In [ ]:
import os, sys
import scipy.misc
from glob import glob
import numpy as np
import random 
import shutil 
import cv2
import matplotlib.pyplot as plt
from keras.models import load_model
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization


base_path = os.path.abspath("../")

sys.path.append(base_path)
import helpers

In [ ]:
def predict(model, img):
    
    if img.shape[0] != 256:
        img = scipy.misc.imresize(img, (256,256))
    
    # Normalize image as the trained distribution
    img = img/127.5 - 1.
    
    img = np.expand_dims(img, axis=0)
    img = model.predict(img)
    img = np.squeeze(img, axis=0)
    
    # Rescale to [0,1]
    img = 0.5 * img + 0.5 
    # Comment if display
    #img = img * 255.0
    return img


def load_img(path):
    # read image
    img = scipy.misc.imread(path, mode='RGB').astype(np.float)
    # predict
    img = predict(model, img)
    return img 


def load_img_both(path):
    # read image
    dim = 256
    img = scipy.misc.imread(path, mode='RGB').astype(np.float)
    img_ = scipy.misc.imresize(img, (dim,dim))
    # predict
    img_pred = predict(model, img)
    return img_, img_pred

In [ ]:
name = "n2c"
base_path = os.path.abspath("../")
model_path = os.path.join(base_path, "models", name)

# Load translation model
model = None
model_name = 'n2c.h5'
model = load_model(os.path.join(model_path, model_name), custom_objects={'InstanceNormalization':InstanceNormalization})
model.summary()

In [ ]:
# Image paths
normal = os.path.join(base_path, "dataset/cxr/train/normal")
pneu = os.path.join(base_path, "dataset/cxr/train/pneumonia")

# Destinations
dest_path1 = os.path.join(base_path, "dataset/gen/from_normal_512")
dest_path2 = os.path.join(base_path, "dataset/gen/from_pneu_512")

helpers.create_directory(dest_path1)
helpers.create_directory(dest_path2)

In [ ]:
normal_path = [os.path.join(normal, x) for x in os.listdir(normal)]
pneu_path = [os.path.join(pneu, x) for x in os.listdir(pneu)]

len(normal_path), len(pneu_path)

In [ ]:
normal_path[:3]

In [ ]:
orig = []
syn = []

import random
random.shuffle(normal_path)

for path in normal_path[:20]:
    img, pred = load_img_both(path)
    orig.append(img)
    syn.append(pred)
    
orig = np.array(orig)
syn = np.array(syn)

In [ ]:
orig.shape, syn.shape

In [ ]:
x = orig[:20]
y = syn[:20]


xa = x[3]
xb = y[3]

dim = 256
a, b = 2, 10
x = np.reshape(x, (a, b, dim, dim, 3))
y = np.reshape(y, (a, b, dim, dim, 3))

from numpy.random import rand
import matplotlib.pyplot as plt

test_data = x
r, c = test_data.shape[0], test_data.shape[1]
cmaps = [['viridis', 'binary'], ['plasma', 'coolwarm'], ['Greens', 'copper']]

heights = [a[0].shape[0] for a in test_data]
widths = [a.shape[1] for a in test_data[0]]

fig_width = 15.  # inches
fig_height = fig_width * sum(heights) / sum(widths)

f, axarr = plt.subplots(r,c, figsize=(fig_width, fig_height),
        gridspec_kw={'height_ratios':heights})

for i in range(r):
    for j in range(c):
        axarr[i, j].imshow(test_data[i][j])
        axarr[i, j].axis('off')
plt.subplots_adjust(wspace=0.02, hspace=0, left=0, right=1, bottom=0, top=1)
plt.savefig('{}/{}.pdf'.format(base_path, "normal_256"), dpi=150)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Normal \u27F6 COVID-19', size=16, y=0.93)
fig.subplots_adjust(wspace=0.02, hspace=0, left=0, right=1, bottom=0, top=1)
ax1.imshow(xa)
ax1.axis('off')
ax2.imshow(xb)
ax2.axis('off')
plt.savefig('{}/{}.pdf'.format(base_path, "single_nvc"), dpi=150)
plt.show()

In [ ]:
test_data = y
r, c = test_data.shape[0], test_data.shape[1]
cmaps = [['viridis', 'binary'], ['plasma', 'coolwarm'], ['Greens', 'copper']]

heights = [a[0].shape[0] for a in test_data]
widths = [a.shape[1] for a in test_data[0]]

fig_width = 15.  # inches
fig_height = fig_width * sum(heights) / sum(widths)

f, axarr = plt.subplots(r,c, figsize=(fig_width, fig_height),
        gridspec_kw={'height_ratios':heights})

for i in range(r):
    for j in range(c):
        axarr[i, j].imshow(test_data[i][j])
        axarr[i, j].axis('off')
plt.subplots_adjust(wspace=0.02, hspace=0, left=0, right=1, bottom=0, top=1)
plt.savefig('{}/{}.pdf'.format(base_path, "normal_covid_256"), dpi=300)
plt.show()

### Save generated images

In [ ]:
ct=1
for path in normal_path[:]:
    img = load_img(path)
    scipy.misc.imsave("{}/{}.jpg".format(dest_path1,ct), img)
    ct+=1

In [ ]:
name = "p2c"
base_path = os.path.abspath("../")
model_path = os.path.join(base_path, "models", name)

# Load translation model
model = None
model_name = 'p2c.h5'
model = load_model(os.path.join(model_path, model_name), custom_objects={'InstanceNormalization':InstanceNormalization})
model.summary()

### Save generated images

In [ ]:
ct=1
for path in pneu_path[:]:
    img = load_img(path)
    scipy.misc.imsave("{}/{}_g2.jpg".format(dest_path2,ct), img)
    ct+=1

In [ ]:
orig = []
syn = []

import random
random.shuffle(pneu_path)

for path in pneu_path[:25]:
    img, pred = load_img_both(path)
    orig.append(img)
    syn.append(pred)
    
orig = np.array(orig)
syn = np.array(syn)

In [ ]:
x = orig[:20]
y = syn[:20]

xa = x[11]
xb = y[11]

dim = 256
a, b = 2, 10
x = np.reshape(x, (a, b, dim, dim, 3))
y = np.reshape(y, (a, b, dim, dim, 3))

from numpy.random import rand
import matplotlib.pyplot as plt

test_data = x
r, c = test_data.shape[0], test_data.shape[1]
cmaps = [['viridis', 'binary'], ['plasma', 'coolwarm'], ['Greens', 'copper']]

heights = [a[0].shape[0] for a in test_data]
widths = [a.shape[1] for a in test_data[0]]

fig_width = 15.  # inches
fig_height = fig_width * sum(heights) / sum(widths)

f, axarr = plt.subplots(r,c, figsize=(fig_width, fig_height),
        gridspec_kw={'height_ratios':heights})

for i in range(r):
    for j in range(c):
        axarr[i, j].imshow(test_data[i][j])
        axarr[i, j].axis('off')
plt.subplots_adjust(wspace=0.02, hspace=0, left=0, right=1, bottom=0, top=1)
plt.savefig('{}/{}.pdf'.format(base_path, "pneu_256"), dpi=300)
plt.show()

In [ ]:
test_data = y
r, c = test_data.shape[0], test_data.shape[1]
cmaps = [['viridis', 'binary'], ['plasma', 'coolwarm'], ['Greens', 'copper']]

heights = [a[0].shape[0] for a in test_data]
widths = [a.shape[1] for a in test_data[0]]

fig_width = 15.  # inches
fig_height = fig_width * sum(heights) / sum(widths)

f, axarr = plt.subplots(r,c, figsize=(fig_width, fig_height),
        gridspec_kw={'height_ratios':heights})

for i in range(r):
    for j in range(c):
        axarr[i, j].imshow(test_data[i][j])
        axarr[i, j].axis('off')
plt.subplots_adjust(wspace=0.02, hspace=0, left=0, right=1, bottom=0, top=1)
plt.savefig('{}/{}.pdf'.format(base_path, "pneu_covid_256"), dpi=300)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Pneumonia \u27F6 COVID-19', size=16, y=0.93)
fig.subplots_adjust(wspace=0.02, hspace=0, left=0, right=1, bottom=0, top=1)
ax1.imshow(xa)
ax1.axis('off')
ax2.imshow(xb)
ax2.axis('off')
plt.savefig('{}/{}.pdf'.format(base_path, "single_pvc"), dpi=150)
plt.show()